In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using StatsBase
include("utils.jl")
include("kmeans_moments_lattice.jl")
using .LatticeGeneration
using JLD2
using SDDP, HiGHS, Plots, XLSX, Mustache, PrettyTables

In [13]:
includet("goal_investing_sddp_model.jl")
using .GoalInvestingSDDP

In [29]:
scenario_grid = load_object("scenario_grid.jld2")
println(size(scenario_grid.probs))
println(size(scenario_grid.states))

(16,)
(15, 9, 3)


In [15]:
graph = SDDP.MarkovianGraph(scenario_grid.probs[1:end-1])

Root
 (0, 1)
Nodes
 (1, 1)
 (2, 1)
 (2, 2)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (2, 7)
 (2, 8)
 (2, 9)
 (3, 1)
 (3, 2)
 (3, 3)
 (3, 4)
 (3, 5)
 (3, 6)
 (3, 7)
 (3, 8)
 (3, 9)
 (4, 1)
 (4, 2)
 (4, 3)
 (4, 4)
 (4, 5)
 (4, 6)
 (4, 7)
 (4, 8)
 (4, 9)
 (5, 1)
 (5, 2)
 (5, 3)
 (5, 4)
 (5, 5)
 (5, 6)
 (5, 7)
 (5, 8)
 (5, 9)
 (6, 1)
 (6, 2)
 (6, 3)
 (6, 4)
 (6, 5)
 (6, 6)
 (6, 7)
 (6, 8)
 (6, 9)
 (7, 1)
 (7, 2)
 (7, 3)
 (7, 4)
 (7, 5)
 (7, 6)
 (7, 7)
 (7, 8)
 (7, 9)
 (8, 1)
 (8, 2)
 (8, 3)
 (8, 4)
 (8, 5)
 (8, 6)
 (8, 7)
 (8, 8)
 (8, 9)
 (9, 1)
 (9, 2)
 (9, 3)
 (9, 4)
 (9, 5)
 (9, 6)
 (9, 7)
 (9, 8)
 (9, 9)
 (10, 1)
 (10, 2)
 (10, 3)
 (10, 4)
 (10, 5)
 (10, 6)
 (10, 7)
 (10, 8)
 (10, 9)
 (11, 1)
 (11, 2)
 (11, 3)
 (11, 4)
 (11, 5)
 (11, 6)
 (11, 7)
 (11, 8)
 (11, 9)
 (12, 1)
 (12, 2)
 (12, 3)
 (12, 4)
 (12, 5)
 (12, 6)
 (12, 7)
 (12, 8)
 (12, 9)
 (13, 1)
 (13, 2)
 (13, 3)
 (13, 4)
 (13, 5)
 (13, 6)
 (13, 7)
 (13, 8)
 (13, 9)
 (14, 1)
 (14, 2)
 (14, 3)
 (14, 4)
 (14, 5)
 (14, 6)
 (14, 7)
 (14, 8)
 

In [31]:
data_source = DataFrame(XLSX.readtable("./goal_data.xlsx", "goal_data_single_goal")) 
data = round.(data_source[1:16,:],digits=3)

Row,t,minimum_limit,acceptable_limit,desired_limit,inflows,priority,minimum_utility,acceptable_utility,desired_utility,above_desired_utility,cashflows
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,6.0,8.0,9.0,0.0,0.067,0.278,0.167,0.067,0.001,-9.0
2,2.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
3,3.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
4,4.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
5,5.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
6,6.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
7,7.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
8,8.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
9,9.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0


In [32]:
model_data = GoalsData3(
    data.minimum_limit,
    data.acceptable_limit,
    data.desired_limit,
    data.inflows,
    data.minimum_utility,
    data.acceptable_utility,
    data.desired_utility,
    data.above_desired_utility,
    35.0,
    0.0029
    )

GoalsData3([6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 81.0], [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 128.0], [9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 159.0], [0.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0], Float32[0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.309], Float32[0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.106], Float32[0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.032], Float32[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 35.0, 0.0029f0)

In [35]:
model = asset_management_alm(model_data, scenario_grid)
SDDP.train(model; iteration_limit = 600, log_frequency = 50)


-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 136
  state variables : 3
  scenarios       : 2.05891e+14
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [18, 18]
  AffExpr in MOI.EqualTo{Float64}         : [5, 5]
  VariableRef in MOI.GreaterThan{Float64} : [15, 15]
  VariableRef in MOI.LessThan{Float64}    : [3, 4]
numerical stability report
  matrix range     [9e-01, 1e+00]
  objective range  [1e-03, 1e+00]
  bounds range     [1e+00, 8e+01]
  rhs range        [1e+01, 4e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-----------------------------------------

In [23]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    2000,
    # A list of names to record the values of.
    [:assets, :minimum, :acceptable, :desired, :above_desired, :consumption],
    skip_undefined_variables=true
)

println("Lower bounds", SDDP.calculate_bound(model))

objectives = map(simulations) do simulation
    return sum(stage[:stage_objective] for stage in simulation)
end

μ, ci = SDDP.confidence_interval(objectives)
println("Confidence interval: ", μ, " ± ", ci)

Lower bounds-60.118447720028804
Confidence interval: -60.02521108539734 ± 0.0872608721310038


In [47]:
n_scenarios = 2000
n_stages = 16
n_assets = 3

consumption = zeros(n_scenarios,n_stages)
assets = zeros(n_assets,n_scenarios,n_stages)

for (i, scenario) in enumerate(simulations)
    consumption[i,:] = [node[:consumption] for node in scenario]
    for a in 1:3
        assets[a,i,:] = [node[:assets][a].out for node in scenario]
    end
end


In [20]:
function calculate_perc(data, perc = [0.05, 0.25, 0.5, 0.75, 0.95])
    n_scenarios, n_stages = size(data)
    
    result = zeros(n_stages, length(perc)) 

    for t in 1:n_stages
       result[t,:] = quantile(data[:,t], perc)
    end

    return DataFrame(round.(result,digits=2),string.(perc))
  
end

calculate_perc (generic function with 2 methods)

In [38]:
calculate_perc(consumption)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,8.0,8.0,8.0,8.0,8.0
2,8.0,8.0,8.0,8.0,8.0
3,8.0,8.0,8.0,8.0,9.0
4,8.0,8.0,8.0,8.8,9.0
5,8.0,8.0,8.0,9.0,9.0
6,8.0,8.0,8.0,9.0,9.0
7,8.0,8.0,8.0,9.0,9.0
8,8.0,8.0,8.16,9.0,9.0
9,8.0,8.0,9.0,9.0,9.0


In [28]:
scenario =13
t = 5
simulations[scenario][t][:assets]

3-element Vector{SDDP.State{Float64}}:
 SDDP.State{Float64}(36.839455387467225, 38.98843049006662)
 SDDP.State{Float64}(0.0, 0.0)
 SDDP.State{Float64}(0.0, 0.0)

In [49]:
for a in 1:3
    display(calculate_perc(assets[a,:,:]))
end

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,26.92,26.92,26.92,26.92,26.92
2,29.68,30.83,32.71,35.42,41.87
3,20.33,35.29,37.66,42.62,48.83
4,31.91,37.6,41.94,46.23,53.3
5,35.06,40.62,46.31,52.4,61.41
6,33.52,43.85,50.57,58.28,69.16
7,39.78,48.28,55.01,63.3,72.56
8,43.96,52.64,60.45,68.36,80.74
9,44.58,56.16,65.06,72.95,87.65


Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.18
3,0.0,0.0,0.0,0.0,22.21
4,0.0,0.0,0.0,5.73,15.74
5,0.0,0.0,0.0,7.35,19.85
6,0.0,0.0,0.0,8.4,32.55
7,0.0,0.0,0.0,12.31,33.11
8,0.0,0.0,0.0,14.22,32.73
9,0.0,0.0,0.0,17.24,41.79


Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0
